# Reading files

In [1]:
import your

your.__version__

## Loading data

Download and extract a sample filterbank. Here we use the filterbanks containing [FRB180417](https://doi.org/10.1093/mnras/stz2574).

In [2]:
import logging

logging_format = "%(asctime)s - %(funcName)s -%(name)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=logging_format)

In [3]:
import os
import tempfile
import pylab as plt
from urllib.request import urlretrieve

In [4]:
temp_dir = tempfile.TemporaryDirectory()
download_path = str(temp_dir.name) + "/FRB180417.fil"
url = "https://zenodo.org/record/3905426/files/FRB180417.fil"
urlretrieve(
    url,
    download_path,
)
fil_file = download_path

### Using the Context Manager

It is recommended to use the `Your` object as a context manager to ensure that files are properly closed after use. This can be done using a `with` statement:

```python
import your
import os # Add os import for path joining if using data files
from your.utils.misc import check_file_exist # To check if example files exist

# Define data_dir for example files, assuming this notebook is in examples/
# and test data is in examples/data/
# Adjust if your data directory structure is different or if files are elsewhere.
data_dir = "data" # Relative path to data from the notebook's location

# Example with a filterbank file
fil_file_example = os.path.join(data_dir, "small.fil") # Using an actual small test file
print(f"Checking for example file: {fil_file_example}")
try:
    check_file_exist(fil_file_example) # Utility to ensure file exists before trying to open
    print(f"Using filterbank file: {fil_file_example}")
    with your.Your(fil_file_example) as y_obj:
        header = y_obj.your_header
        print(f"Source: {header.source_name}")
        data = y_obj.get_data(nstart=0, nsamp=10) # Reading only 10 samples
        # File and mmap object are automatically closed
        print(f"Inside 'with' block: Filterbank file object open status: {not y_obj.formatclass.fp.closed}")
    print(f"After 'with' block: Filterbank file object closed: {y_obj.formatclass.fp.closed}")
    if hasattr(y_obj.formatclass._mmdata, 'closed'):
         print(f"After 'with' block: Filterbank mmap object closed: {y_obj.formatclass._mmdata.closed}")
    else:
        try:
            y_obj.formatclass._mmdata.size()
            print("After 'with' block: Filterbank mmap object appears to be open (size() did not fail)")
        except ValueError:
            print("After 'with' block: Filterbank mmap object closed (size() failed as expected)")
except FileNotFoundError as e:
    print(e)
    print("Skipping filterbank context manager example as file is not found.")


# Example with a PSRFITS file
fits_file_example = os.path.join(data_dir, "small.fits") # Using an actual small test file
print(f"\nChecking for example file: {fits_file_example}")
try:
    check_file_exist(fits_file_example)
    print(f"Using PSRFITS file: {fits_file_example}")
    with your.Your(fits_file_example) as y_obj:
        print(f"Center frequency: {y_obj.your_header.center_freq}")
        # FITS file is automatically closed
        print(f"Inside 'with' block: PSRFITS file object open status: {not y_obj.formatclass.fits._file.closed}")
    print(f"After 'with' block: PSRFITS file object closed: {y_obj.formatclass.fits._file.closed}")
except FileNotFoundError as e:
    print(e)
    print("Skipping PSRFITS context manager example as file is not found.")

# Example with a list of (dummy) PSRFITS files
# Using dummy names as setting up multiple real files for a notebook example can be complex.
# The tests cover multi-file scenarios with real files.
print("\nUsing a list of (dummy) PSRFITS files for context manager example:")
# Note: The following will likely fail if these files don't exist.
# For a runnable example, these should be paths to actual files.
# For documentation, placeholder names are fine if clearly marked.
dummy_fits_files = ["dummy_file1.fits", "dummy_file2.fits"]
print(f"Attempting to use: {dummy_fits_files}. These are placeholders and will likely cause an error if they don't exist.")
try:
    with your.Your(dummy_fits_files) as y_obj:
        print(f"Total native spectra (from list): {y_obj.your_header.native_nspectra}")
        # All PSRFITS files associated are handled by the context manager
    print("PSRFITS files in list are handled by context manager.")
except Exception as e:
    print(f"Error with dummy PSRFITS file list example (as expected if files don't exist): {e}")
```

## Unified Reader

We can now read the file using [Your](https://thepetabyteproject.github.io/your/your/) as,

In [5]:
your_object = your.Your(fil_file)

Similarly using `your` we can read 
* list of psrfits files,
* single psrfits file,
* single filterbank file.

## Unified Header

Regardless of the file source, each `your` object has a unified header which can be accessed as follows. [your_header](https://thepetabyteproject.github.io/your/your/#header) displays all the attributes. These can be accessed as `your_object.your_header.attribute`, e.g. to get the sampling interval use `your_object.your_header.tsamp`.

In [6]:
print(your_object.your_header)

## Reading data

You can read the data in the file using the [get_data](https://thepetabyteproject.github.io/your/your/#get_data) method. The output is a numpy array with shape (nsamp, nchans).

In [7]:
data = your_object.get_data(nstart=0, nsamp=4096)
data.shape

And display the data!

In [8]:
plt.figure(figsize=(8, 6))
plt.imshow(data.T, aspect="auto")
plt.xlabel("Time Samples")
plt.ylabel("Frequency Channels")
plt.colorbar()

## Bandpass

You can create the bandpass of the entire file or a few specific spectra by using [your_object.bandpass](https://thepetabyteproject.github.io/your/your/#bandpass)

In [9]:
bandpass = your_object.bandpass()

And plot and save the bandpass using the [save_bandpass](https://thepetabyteproject.github.io/your/utils/plotter/#save_bandpass-function) function from [your.utils.plotter](https://thepetabyteproject.github.io/your/utils/plotter/).

In [10]:
from your.utils.plotter import save_bandpass

In [11]:
save_bandpass(your_object, bandpass)